<a href="https://colab.research.google.com/github/linnbogren/big_data_project/blob/main/PBD_Project_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# mount drive - Only if you are working with Colab, discard otherwise

from google.colab import drive
drive.mount('/content/drive')

data_folder_path = '/content/drive/My Drive/KTH/utbyte/Big Data/project_big_data/Project_Data'
features_path = data_folder_path + '/Features'
audio_path = data_folder_path + '/Audio'
video_frames_path = data_folder_path + '/Video_Frames'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import os
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import gdown

# **Visual Data**

In [9]:
#List all Pickle (.pkl) files
pklfiles = [f for f in os.listdir(features_path) if os.path.isfile(os.path.join(features_path, f)) and f.endswith('_visual.pkl')]
print(pklfiles)

['cdu-ps_visual.pkl', 'chega-be_visual.pkl', 'ad-livre_visual.pkl', 'ad-cdu_visual.pkl', 'ad-be_visual.pkl', 'chega-cdu_visual.pkl', 'be-ps_visual.pkl', 'ad-il_visual.pkl', 'cdu-be_visual.pkl', 'ad-pan_visual.pkl', 'ad-ps_visual.pkl', 'chega-il_visual.pkl', 'ad-chega_visual.pkl', 'pan-ps_visual.pkl', 'pan-chega_visual.pkl', 'il-ps_visual.pkl', 'livre-ps_visual.pkl', 'livre-chega_visual.pkl', 'pan-cdu_visual.pkl', 'il-cdu_visual.pkl', 'il-be_visual.pkl', 'chega-ps_visual.pkl', 'livre-be_visual.pkl', 'pan-livre_visual.pkl', 'pan-il_visual.pkl', 'livre-il_visual.pkl', 'livre-cdu_visual.pkl', 'pan-be_visual.pkl']


In [ ]:
# Load one
video = 'ad-ps'
data = pd.read_pickle(os.path.join(features_path, video + '_visual.pkl'))

# Print first lines
data.head()

In [ ]:
#Access one frame
data.iloc[0]

In [ ]:
#Lets visualize some features in the images - Detections
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

frame = 2012

frame_id = data.iloc[frame]['filename']
print(frame_id)

img = Image.open(os.path.join(video_frames_path, video, frame_id))
plt.imshow(img)
plt.show()

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.imshow(img)

print(data.iloc[frame]['detections'])

for bbox in data.iloc[frame]['detections']:

    # Create a rectangle patch
    rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')

    # Add patch to the image
    ax.add_patch(rect)

    ax.text(bbox[0], bbox[1], f'{bbox[4]} ({100*bbox[5]:.1f}%)',
            verticalalignment='top',
            color='r')

plt.show()

In [ ]:
# Poses
poses = np.array(data.iloc[frame]['poses'])

print(poses.shape) # 1 pose, 33 keypoints, 5 values [x,y,z,visibility,presence]

#With visibility and presence thresholds
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.imshow(img)

width, height = img.size
for pose in np.array(data.iloc[frame]['poses']):

    # convert normalized landmark to pixel position
    x = np.floor(pose[:,0] * width)
    y = np.floor(pose[:,1] * height)

    for i in range(pose.shape[0]):
        if pose[i,3]>.5 and pose[i,4]>.5: #visibility and presence thresholds
            plt.scatter(x[i], y[i], color='w')

plt.show()

In [ ]:
# Facial emotion recognition

fers = data.iloc[frame]['fer']

print(fers[0].keys())



In [ ]:
print(fers[0]['location'])
print(fers[0]['embedding'])

print(fers[0]['emotion'])

print(fers[0]['logits'])

In [ ]:
#Visualize emotions and detected faces
idx_to_class={0: 'Anger', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happiness', 5: 'Neutral', 6: 'Sadness', 7: 'Surprise'}

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.imshow(img)

width, height = img.size

fers = data.iloc[frame]['fer']
print(fers)

for fer in fers:

    x = fer['location'][0]
    y = fer['location'][2]
    width = fer['location'][1] - fer['location'][0]
    height = fer['location'][3] - fer['location'][2]

    print(fer['emotion'])

    # Create a rectangle patch
    rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='g', facecolor='none')

    # Add patch to the image
    ax.add_patch(rect)

    ax.text(x, y, fer['emotion'],
            verticalalignment='top',
            color='g')

plt.show()

# **Audio & Speech**

In [ ]:
#List all Pickle (.pkl) files

pklfiles = [f for f in os.listdir(features_path) if os.path.isfile(os.path.join(features_path, f)) and f.endswith('_audio.pkl')]
print(pklfiles)

In [ ]:
# Load one
video = 'ad-ps'
data_audio = pd.read_pickle(os.path.join(features_path, video + '_audio.pkl'))

# Print first lines
data_audio.head()

In [ ]:
print(data_audio.keys()) #Check feature names

In [ ]:
#Align with the video frame (second 1712)
print(data_audio['time stamp'][19:25])
print(data_audio['duration'][19:25])

In [ ]:
print(data_audio.iloc[19])

In [ ]:
#Lets listen
from scipy.io import wavfile
from IPython.display import Audio

samplerate, audio = wavfile.read(os.path.join(audio_path, video + '.wav'))

sample = audio[int(data_audio.iloc[19]['time stamp']*samplerate):int(data_audio.iloc[19]['time stamp']+data_audio.iloc[19]['duration'])*samplerate,0]

display(Audio(sample,rate=samplerate))

In [ ]:
#List all Pickle (.pkl) files

pklfiles = [f for f in os.listdir(features_path) if os.path.isfile(os.path.join(features_path, f)) and f.endswith('_speech.pkl')]
print(pklfiles)

In [ ]:
# Load one
video = 'ad-ps'
data_speech = pd.read_pickle(os.path.join(features_path, video + '_speech.pkl'))

# Print first lines
data_speech.head()

In [ ]:
#Verify the transcritp
transcript = data_speech.iloc[19]['transcript']
print(transcript)

# **Text**

In [ ]:
#Load the single file
# Load one
data_dictionary = pd.read_pickle(os.path.join(features_path, 'dictionary_text.pkl'))

# Print first lines
data_dictionary.head()

In [ ]:
#How to explore the dictionary and the transcripts? - Try using Bag-of-Words
#Cool demo at https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/text-analysis/notebooks/Counting%20words%20with%20scikit-learn's%20CountVectorizer.ipynb
from sklearn.feature_extraction.text import CountVectorizer

corpus = [transcript]

vectorizer = CountVectorizer(vocabulary = data_dictionary.index)
count_matrix = vectorizer.transform(corpus)
bow = vectorizer.get_feature_names_out()
count_array = count_matrix.toarray()

print(count_array.shape)
print(data_dictionary.index.shape)

words = np.where(np.squeeze(count_array) != 0)

#See words that were found
print(bow[words])

In [ ]:
#For a full video
texts = data_speech['transcript']

corpus = []

for text in texts:
  corpus.append(text)

count_matrix = vectorizer.transform(corpus)
bow = vectorizer.get_feature_names_out()
count_array = count_matrix.toarray()

print(count_array.shape)
print(data_dictionary.index.shape)

for i in range(count_array.shape[0]):
  words = np.where(np.squeeze(count_array[i,:]) != 0)

  print(bow[words])

In [ ]:
#Load the single file
# Load one
data_pulsometer = pd.read_pickle(os.path.join(features_path, 'pulsometer_text.pkl'))

# Print first lines
data_pulsometer.head()

In [ ]:
#Load the single file
# Load one
data_grades = pd.read_pickle(os.path.join(features_path, 'grades_text.pkl'))

# Print first lines
data_grades.head()